In [91]:
import numpy as np
import sys

def expand_matrix(lat,
                  lon,
                  resolution = None):
    
    if len(lat.shape) == 0:
        lat = lat.reshape((1, 1))
        lon = lon.reshape((1, 1))
        m, n = 1, 1
    elif len(lat.shape) == 1:
        lat = lat.reshape((1, -1))
        lon = lon.reshape((1, -1))
        m, n = lat.shape
    elif len(matrix.shape) == 2:
        m, n = lat.shape
    else:
        sys.exit("dimension of input matrix is more than 2 for lat and lon inputs")
    print(m, n)
    # Create a new (m+1) x (n+1) matrix filled with zeros
    expanded_lat = np.zeros((m+2, n+2), dtype=lat.dtype)
    expanded_lon = np.zeros((m+2, n+2), dtype=lon.dtype)
    # Embed the original matrix in the center of the new matrix
    expanded_lat[1:m+1, 1:n+1] = lat
    expanded_lon[1:m+1, 1:n+1] = lon

    print(expanded_lat)
    print(expanded_lon)
    print(resolution)

    if (m == 1 or n == 1) and (resolution is None):
        sys.exit("user should provide approximate resolution of grids")
    
    if m == 1 and n == 1: # only one grid
        # lat
        expanded_lat [:,:] = expanded_lat [1,1]
        print(expanded_lat)
        diff = np.array([[resolution, resolution, resolution],[0, 0, 0],[-resolution, -resolution, -resolution]])
        expanded_lat = expanded_lat + diff
        # lon
        expanded_lon [:,:] = expanded_lon [1,1]
        diff = np.array([[-resolution, 0, resolution],[-resolution, 0, resolution],[-resolution, 0, resolution]])
        expanded_lon = expanded_lon + diff
        print(expanded_lon)
    
    if 2 <= n and m == 1: # 1 row of grids
        #
        is_ascending = False
        is_ascending = np.all(np.diff(lon.flatten()) > 0)
        is_descending = False
        is_descending = np.all(np.diff(lon.flatten()) < 0)

        if (is_ascending and is_descending) or (not is_ascending and not is_descending):
            sys.exit("EASYMORE cannot decide if the lon values of source nc are ascending or descending")
        
        is_uniform = False
        is_uniform = (np.all(np.abs(np.diff(lon.flatten()) + resolution) < 0.00001) or 
                      np.all(np.abs(np.diff(lon.flatten()) - resolution) < 0.00001))
        if not is_uniform:
            print("EASYMORE detects that lon values of source nc are not uniformly spaces as specified by resolution and tolerance")
            print("EASYMORE continue creation of the shapefile from source nc file with approximation")
        
        # lat
        expanded_lat[1,0] = expanded_lat[1,1]
        expanded_lat[1,-1] = expanded_lat[1,-2]
        expanded_lat[0,:] = expanded_lat [1,:]
        expanded_lat[2,:] = expanded_lat [1,:]
        diff = np.tile(np.array([[-resolution], [0], [resolution]]), (1, n+2))
        expanded_lat = expanded_lat + diff
        # lon
        expanded_lon[0,:] = expanded_lon [1,:]
        expanded_lon[2,:] = expanded_lon [1,:]
        expanded_lon[:,0] = expanded_lon[:,1]
        expanded_lon[:,-1] = expanded_lon[:,-2]
        diff = expanded_lon * 0.00
        if is_ascending:
            diff [:,0] = -resolution
            diff [:,-1] = resolution
        else:
            diff [:,0] = resolution
            diff [:,-1] = -resolution
        expanded_lon = expanded_lon + diff
    
    if 2 <= m and n == 1: # 1 column of grids

        #
        is_ascending = False
        is_ascending = np.all(np.diff(lat.flatten()) > 0)
        is_descending = False
        is_descending = np.all(np.diff(lat.flatten()) < 0)

        if (is_ascending and is_descending) or (not is_ascending and not is_descending):
            sys.exit("EASYMORE cannot decide if the lat values of source nc are ascending or descending")
        
        is_uniform = False
        is_uniform = (np.all(np.abs(np.diff(lat.flatten()) + resolution) < 0.00001) or 
                      np.all(np.abs(np.diff(lat.flatten()) - resolution) < 0.00001))
        if not is_uniform:
            print("EASYMORE detects that lat values of source nc are not uniformly spaces as specified by resolution and tolerance")
            print("EASYMORE continue creation of the shapefile from source nc file with approximation")

        # lat
        expanded_lat[0,1] = expanded_lat[1,1]
        expanded_lat[-1,1] = expanded_lat[-2,1]
        expanded_lat[:,0] = expanded_lat [:,1]
        expanded_lat[:,2] = expanded_lat [:,1]
        diff = expanded_lat * 0.00
        if is_ascending:
            diff [0,:] = -resolution
            diff [-1,:] = resolution
        else:
            diff [0,:] = resolution
            diff [-1,:] = -resolution
        expanded_lat = expanded_lat + diff
        
        # lon
        expanded_lon[0,1] = expanded_lon[1,1]
        expanded_lon[-1,1] = expanded_lon[-2,1]
        expanded_lon[:,0] = expanded_lon [:,1]
        expanded_lon[:,2] = expanded_lon [:,1]
        diff = expanded_lon * 0.00
        diff [:,0] = -resolution
        diff [:,-1] = resolution
        expanded_lon = expanded_lon + diff
        
    if (2 <= m) and (2<= n):
        
        # create expanded lat
        expanded_lat [:, 0]  = expanded_lat [:, 1] + (expanded_lat [:, 1] - expanded_lat [:, 2]) # populate left column
        expanded_lat [:,-1]  = expanded_lat [:,-2] + (expanded_lat [:,-2] - expanded_lat [:,-3]) # populate right column
        expanded_lat [0, :]  = expanded_lat [1, :] + (expanded_lat [1, :] - expanded_lat [2, :]) # populate top row
        expanded_lat [-1,:]  = expanded_lat [-2,:] + (expanded_lat [-2,:] - expanded_lat [-3,:]) # populate bottom row
        # create expanded lat
        expanded_lon [:, 0]  = expanded_lon [:, 1] + (expanded_lon [:, 1] - expanded_lon [:, 2]) # populate left column
        expanded_lon [:,-1]  = expanded_lon [:,-2] + (expanded_lon [:,-2] - expanded_lon [:,-3]) # populate right column
        expanded_lon [0, :]  = expanded_lon [1, :] + (expanded_lon [1, :] - expanded_lon [2, :]) # populate top row
        expanded_lon [-1,:]  = expanded_lon [-2,:] + (expanded_lon [-2,:] - expanded_lon [-3,:]) # populate bottom row
    
    return expanded_lat, expanded_lon

# #
# lat = np.array([2])
# lon = np.array([3])

# #
# lat = np.array([2,2])
# lon = np.array([6,6.5])

# #
# lat = np.array([2,2])
# lon = np.array([6,5.5])

# #
# lat = np.array([[1.5],[1]])
# lon = np.array([[3],[3]])

#
lat = np.array([[1.5],[2]])
lon = np.array([[3],[3]])

#
lat = np.array([[1.5, 1.5],[2.0, 2.0]])
lon = np.array([[3.0, 3.5],[3.0, 3.5]])

#
lat = np.array([[2.0, 2.0],[1.5, 1.5]])
lon = np.array([[3.5, 3.0],[3.5, 3.0]])

expanded_lat, expanded_lon = expand_matrix (lat, lon, resolution = 0.5)

print(expanded_lat)
print(expanded_lon)

2 2
[[0.  0.  0.  0. ]
 [0.  2.  2.  0. ]
 [0.  1.5 1.5 0. ]
 [0.  0.  0.  0. ]]
[[0.  0.  0.  0. ]
 [0.  3.5 3.  0. ]
 [0.  3.5 3.  0. ]
 [0.  0.  0.  0. ]]
0.5
[[2.5 2.5 2.5 2.5]
 [2.  2.  2.  2. ]
 [1.5 1.5 1.5 1.5]
 [1.  1.  1.  1. ]]
[[4.  3.5 3.  2.5]
 [4.  3.5 3.  2.5]
 [4.  3.5 3.  2.5]
 [4.  3.5 3.  2.5]]


In [9]:
np.ones([3])

array([1., 1., 1.])

In [67]:
[[1.  1.  1.  1. ]
 [1.5 1.5 1.5 1.5]
 [2.  2.  2.  2. ]
 [2.5 2.5 2.5 2.5]]
[[2.5 3.  3.5 4. ]
 [2.5 3.  3.5 4. ]
 [2.5 3.  3.5 4. ]
 [2.5 3.  3.5 4. ]]

True True


In [68]:
diff = np.diff(lat)
diff

array([], shape=(2, 0), dtype=int64)